In [0]:
from googleapiclient.discovery import build

In [0]:
from oauth2client.service_account import ServiceAccountCredentials

In [0]:
import httplib2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
import argparse
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'C:/client_secret_442504306103-saccqakerkn8epihqmp8ctj2mq1ik08a.apps.googleusercontent.com.json'
VIEW_ID = "194249459"

In [0]:
def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      KEY_FILE_LOCATION, scope=SCOPES,
      message=tools.message_if_missing(KEY_FILE_LOCATION))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', http=http)
  return analytics


def get_report(analytics):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
            'pageSize': 100000,
          'dateRanges': [{'startDate': '2019-06-01', 'endDate': '2019-10-31'}],
            'dimensions': [{'name':'ga:date'},
                           {'name':'ga:pagePath'},
                           {'name':'ga:fullReferrer'},
                           {'name':'ga:userType'}
                          ],
          'metrics': [{'expression': 'ga:users'},
                     {'expression': 'ga:sessions'},
                     {'expression': 'ga:bounceRate'},
                     {'expression': 'ga:avgSessionDuration'},
                     {'expression': 'ga:pageviews'},
                     {'expression': 'ga:uniquePageviews'},
                    {'expression': 'ga:avgTimeOnPage'},
                     {'expression': 'ga:exitRate'},
                     {'expression': 'ga:organicSearches'}],
            }]}
  ).execute()

def print_response(response):
    data = response['reports'][0]['data']['rows']
    date=[v['dimensions'][0] for v in data]
    page=[v['dimensions'][1] for v in data]
    referrer= [v['dimensions'][2] for v in data]
    users = [v['metrics'][0]['values'][0] for v in data]
    sessions = [v['metrics'][0]['values'][1] for v in data]
    bouncerate = [v['metrics'][0]['values'][2] for v in data]
    avgsessiondur = [v['metrics'][0]['values'][3] for v in data]
    pageview = [v['metrics'][0]['values'][4] for v in data]
    uniquepageview = [v['metrics'][0]['values'][5] for v in data]
    avgtimeonpage = [v['metrics'][0]['values'][6] for v in data]
    exitrate = [v['metrics'][0]['values'][7] for v in data]
    organicsearch = [v['metrics'][0]['values'][8] for v in data]

    df1 = pd.DataFrame()
    df1['Date']=date
    df1['Date'] = pd.to_datetime(df1.Date)
    df1['PagePath'] = page
    df1['Referrer'] = referrer
    df1['User'] = users
    df1['Session'] = sessions
    df1['BounceRate'] = bouncerate
    df1['AvgsessionDur'] = avgsessiondur
    df1['Pageview'] = pageview
    df1['UniquePageview'] = uniquepageview
    df1['AvgtimeonPage'] = avgtimeonpage
    df1['ExitRate'] = exitrate
    df1['OrganicSearch'] = organicsearch
    print(df1.head())
    df1.to_csv('apm_data_cont_v3.csv',sep='\t',index=False,header=True)

def main():

  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  print_response(response)

if __name__ == '__main__':
  main()

        Date                                  PagePath  \
0 2019-06-01                      mychariotenergy.com/   
1 2019-06-01                      mychariotenergy.com/   
2 2019-06-01                      mychariotenergy.com/   
3 2019-06-01  mychariotenergy.com/?ZipCode=&Promocode=   
4 2019-06-01           mychariotenergy.com/businesses/   

                     Referrer User Session BounceRate AvgsessionDur Pageview  \
0                    (direct)    1       1        0.0          30.0        1   
1                    (direct)    1       1      100.0           0.0        1   
2                      google    1       1        0.0         771.0        1   
3                      google    1       0        0.0           0.0        2   
4  chariotstage.wpengine.com/    1       1      100.0           0.0        1   

  UniquePageview AvgtimeonPage ExitRate OrganicSearch  
0              1          30.0      0.0             0  
1              1           0.0    100.0             0  
2 